## Introducción a PyTorch (Parte 1)

<a target="_blank" href="https://colab.research.google.com/github/pglez82/DeepLearningWeb/blob/master/labs/notebooks/Introducci%C3%B3n%20a%20PyTorch%20(Parte%201).ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

PyTorch es un framework de aprendizaje profundo desarrollado por Facebook, de **código abierto** y con contribuciones de miles de usuarios. Es una alternativa a otros frameworks como TensorFlow o MXNet. El lenguaje de programación utilizado por este framework es Python (aunque muchas de sus partes están programas en otros lenguajes como C++). En este tutorial, vamos a aprender los fundamentos de PyTorch para que puedas utilizarlo en el resto de prácticas.



#### Primeros pasos
Lo primero consiste en ver si tenemos PyTorch instalado y conocer su versión:

In [1]:
import torch
print(torch.__version__)

2.8.0+cu126


si en la salida anterior ves **cpu** será que estás ejecutando una compilación de PyTorch solo con soporte para CPU y no GPU. Si por el contrario quieres ejecutar PyTorch en una máquina con GPU como Google Colab (o incluso tu propia máquina con GPU y Cuda instalado), la salida de este comando debería indicartelo. Ten en cuenta que la versión con GPU también soporta entrenamientos en la CPU (lo contrario no es cierto).

Un aspecto importante en los experimentos que hagamos será la reproducibilidad de resultados. Establecemos una semilla para que todos los números aleatorios generados sean los mismos ejecución tras ejecución:

In [2]:
torch.manual_seed(2032)

#### Tensores

Los tensores son la pieza clave en cualquier framework de aprendizaje profundo. Son equivalentes a los arrays de Numpy pero tienen ciertas diferencias muy importantes:

1. Los tensores **pueden moverse entre diferentes dispositivos**. Es decir, podemos tener un tensor en CPU y moverlo a la GPU y todos los cálculos realizados con este pasarán a realizarse en este dispositivo.
2. Los tensores están preparados para diferenciar sobre ellos (calcular las derivadas parciales necesarias para aplicar descenso de gradiente).

De todas maneras, una de las principales ventajas de PyTorch es que si sabemos operar con arrays de Numpy, cambiar a hacerlo con tensores será muy sencillo. Vamos a crear nuestro primer Tensor:

In [3]:
x = torch.Tensor(3, 4)
print(x)

tensor([[5.6528e-21, 4.5074e-41, 5.6528e-21, 4.5074e-41],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00,        nan, 1.8788e+31, 1.7220e+22]])


aquí tenemos un tensor de 3x4 de números reales, inicializado alteatoriamente. Podemos mostrar su dimensión:

In [4]:
print(x.shape)

torch.Size([3, 4])


También es posible inicializar tensores con otros valores, como por ejemplo, ceros, unos o valores aleatorios entre 0 y 1:

In [5]:
print(torch.zeros(3,4))
print(torch.ones(3,4))
print(torch.rand(3,4))

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[0.6282, 0.7710, 0.5404, 0.5480],
        [0.0920, 0.3038, 0.9887, 0.5169],
        [0.7733, 0.7820, 0.8844, 0.8440]])


Otra manera de crear un tensor es hacerlo desde un array de numpy existente:

In [6]:
import numpy as np
np_arr = np.array([[1, 2], [3, 4]])
tensor = torch.from_numpy(np_arr)
print(tensor)
print(tensor.shape)

tensor([[1, 2],
        [3, 4]])
torch.Size([2, 2])


En este case se pude ver que como el array era de números enteros, el tensor resultante mantiene este tipo. Siempre podemos ver el **tipo de los elementos de un tensor** con la siguiente instrucción:

In [7]:
print(tensor.dtype)

torch.int64


Ten en cuenta que **el tipo es crítico** ya que nuestra red va a requerir muchísimos parámetros que al final van a ser tensores y la memoria de nuestros dispositivos es limitada. Te recomiendo el siguiente [enlace](https://pytorch.org/docs/stable/tensors.html) para conocer los diferentes tipos y saber cuando ocupa cada uno en memoria.

Además de crear tensores, muchas veces es interesante convertirlos de vuelta a Numpy. Podemos hacerlo de la siguiente manera:

In [8]:
tensor.cpu().numpy()

array([[1, 2],
       [3, 4]])

Ten en cuenta que la llamada cpu() lo que hace es mover el tensor a la cpu (si no está ya). Es importante hacer esta llamada porque para pasar el tensor a numpy tiene que estar en cpu primero.

Para mover tensores de un dispositivo a otro podemos hacerlo de la siguiente manera.

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tensor = tensor.to(device)

obviamente para que esto funcione debemos tener PyTorch instalado con soporte para cuda (sino el tensor se quedará en la cpu). En este caso **cuda:0** indica que queremos mover el tensor a la primera GPU del sistema. Es importante tener en cuenta que `tensor.to` devuelve el tensor en el nuevo dispositivo por tanto debemos recordar guardarlo en una variable para posteriormente poder usarlo.

#### Operaciones con tensores

Existen multitud de operaciones que se pueden realizar con tensores. En el siguiente [enlace](https://pytorch.org/docs/stable/tensors.html#) tienes una descripción completa de todas las operaciones que se pueden realizar. Aquí vamos a describir las más básicas.

In [10]:
t1 = torch.rand(3,2)
t2 = torch.rand(3,2)
suma = t1+t2
print(t1)
print(t2)
print(suma)

tensor([[0.6985, 0.2108],
        [0.1112, 0.1474],
        [0.8116, 0.1859]])
tensor([[0.5345, 0.4596],
        [0.3705, 0.5302],
        [0.5715, 0.7342]])
tensor([[1.2330, 0.6704],
        [0.4817, 0.6776],
        [1.3831, 0.9201]])


ten en cuenta que esta operación crea un nuevo tensor en memoria. En PyTorch es posible también realizar **operaciones sobre los mismos tensores**, para no gastar espacio extra en memoria. Por ejemplo:

In [11]:
t2.add_(t1)

tensor([[1.2330, 0.6704],
        [0.4817, 0.6776],
        [1.3831, 0.9201]])

Tenemos otras operaciones disponibles pero en general, **las operaciones básicas que puedes hacer con Numpy también se pueden hacer con tensores**:

In [12]:
t1 = torch.rand(3,2)
t2 = torch.rand(3,2)
print(t1)
print(t2)
print("Resta:")
print(t1-t2)
print("Multiplicación por un escalar:")
print(t1*3)
print("Multiplicación de matrices elemento a elemento:")
print(t1*t2)
print("Multiplicación de matrices normal:")
#Importante: estamos haciendo la transpuesta de t2 para poder multiplicarlas y que coincidan las dimesiones
print(t1@t2.T)

tensor([[0.4333, 0.4734],
        [0.1678, 0.0093],
        [0.9842, 0.3493]])
tensor([[0.5617, 0.6045],
        [0.8110, 0.5253],
        [0.3047, 0.6035]])
Resta:
tensor([[-0.1284, -0.1311],
        [-0.6432, -0.5160],
        [ 0.6795, -0.2542]])
Multiplicación por un escalar:
tensor([[1.2999, 1.4201],
        [0.5034, 0.0278],
        [2.9526, 1.0479]])
Multiplicación de matrices elemento a elemento:
tensor([[0.2434, 0.2861],
        [0.1361, 0.0049],
        [0.2999, 0.2108]])
Multiplicación de matrices normal:
tensor([[0.5295, 0.6001, 0.4177],
        [0.0999, 0.1410, 0.0567],
        [0.7640, 0.9817, 0.5107]])


#### Cambio de la forma de un tensor

En muchas ocasiones necesitamos cambiar la forma de un tensor, para luego poder operar con él correctamente. Para esto es muy adecuada la función **view**:

In [13]:
#arange crea un tensor con valores desde 0 hasta n-1
t1 = torch.arange(10)
print(t1)

#digamos que queremos una matriz de 5x2
print(t1.view(5,2))

#También podemos inferir dimensiones
print(t1.view(5,-1))

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])
tensor([[0, 1],
        [2, 3],
        [4, 5],
        [6, 7],
        [8, 9]])


es interesante entender que view devuelve un nuevo tensor pero que comparte la estructura interna con el tensor original, es decir, no estamos almacenando los datos de nuevo, sino simplemente **dándoles otra forma**.

#### Indexado
El indexado funciona de igual manera que en Numpy. Veamos algunos ejemplos:

In [14]:
t1 = torch.arange(12).view(3,4)
print(t1)

print("Solo la segunda fila (empieza a contar en cero):")
print(t1[1,:])
print("Solo la última columna:")
print(t1[:,-1])
print("Seleccionar el primer elemento de las dos primeras filas:")
print(t1[:2,0])

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
Solo la segunda fila (empieza a contar en cero):
tensor([4, 5, 6, 7])
Solo la última columna:
tensor([ 3,  7, 11])
Seleccionar el primer elemento de las dos primeras filas:
tensor([0, 4])


### Ejercicios propuestos
1. Navega por la documentación de PyTorch (https://pytorch.org/docs/stable/torch.html) y busca un par de funciones interesantes para operar con tensores que no aparezcan en este notebook.
2. Ejecuta el notebook en Google Colab. Cambia el tipo de dispositivo de tu máquina y trata de ejecutar el código anterior en diferentes dispositivos. Usa el atributo del tensor `device` para conocer en que dispositivo se encuentra.
3. Intenta realizar una operación con dos tensores que se encuentren en dispositivos diferentes. ¿Qué sucede en este caso?
4. Ejecuta el notebook en local, comprueba que todos los tensores están en la CPU.
5. Implementa un perceptrón utilizando tensores en PyTorch. Comprueba que su salida ante una entrada concreta es correcta.
6. Realiza las siguientes operaciones con tensores:
   - Crear Tensores: Crea los siguientes tensores en PyTorch:
      - Un tensor *t_1* de 10 elementos igualmente espaciados entre 0 y 1.
      - Un tensor *t_2* de tamaño 3x3 con valores aleatorios.
      - Un tensor *t_3* de tamaño 2x3x4 con todos sus elementos inicializados a 1.
   - Extrae la segunda fila del tensor *t_2*.
   - Cambia la forma (reshape) de *t_3* a un tensor 2D de tamaño 6x4.
   - Transpón el tensor *t_2* (intercambia filas por columnas).
   - Suma el tensor [1.0, 1.0, 1.0] a la primera fila de *t_2*.
   - Realiza un producto elemento a elemento entre *t_2* y su transpuesta.
   - Realiza un producto matricial entre *t_2* y su transpuesta.
   - Selecciona todos los elementos de la *t_2* que sean mayores que 0.5.
   - Crea un tensor booleano de la misma forma que el *t_2*, que sea True si el elemento es mayor que 0.5, y False en caso contrario.
   - Cuenta cuantos elementos de *t_2* son mayores de que la media de los elementos de *t_2*.


Ejercicio Propuesto 1:

Comando is_tensor -> Devuelve true si el objeto al que aplicamos la función es un tensor.

Comando linspace -> Crea un tensor de una dimensión cuyos valores están igualmente espaciados en un rango dado.

Ejercicio Propuesto 3:



In [17]:
tensor_cpu = torch.rand(3,4)
tensor_gpu = torch.rand(3,4)

tensor_cpu = tensor.to("cpu")
tensor_gpu = tensor.to("cuda:0")

#tensor_gpu.add_(tensor_cpu)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

Como vemos, si ejecutamos la suma en 2 dispositivos diferentes nos da error.

Ejercicio Propuesto 5: (Demasiado movida)

In [18]:
#Creamos los datos de entrada X y las etiquetas y:
X =  torch.rand(5,2)
y = torch.tensor([
    [0],
    [1],
    [1],
    [1],
    [0]
])

In [ ]:
#Creamos el sesgo y la varianza:


Ejercicio Propuesto 6:

In [36]:
#Un tensor t_1 de 10 elementos igualmente espaciados entre 0 y 1.
t_1 = torch.linspace(0,1,10)
print(t_1)

#Un tensor t_2 de tamaño 3x3 con valores aleatorios
t_2 = torch.rand(3,3)
print(t_2)

#Un tensor t_3 de tamaño 2x3x4 con todos sus elementos inicializados a 1.
t_3 = torch.ones(2,3,4)
print(t_3)

tensor([0.0000, 0.1111, 0.2222, 0.3333, 0.4444, 0.5556, 0.6667, 0.7778, 0.8889,
        1.0000])
tensor([[0.8616, 0.6108, 0.5161],
        [0.7833, 0.0530, 0.8470],
        [0.0586, 0.0789, 0.9867]])
tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])


In [45]:
#Extrae la segunda fila del tensor t_2.
print(t_2[1,:])

#Cambia la forma (reshape) de t_3 a un tensor 2D de tamaño 6x4.
print(t_3.view(6,4))

#Transpón el tensor t_2 (intercambia filas por columnas).
t_2 = t_2.t()

#Suma el tensor [1.0, 1.0, 1.0] a la primera fila de t_2.
t_2[0,:].add_(torch.tensor([1.0, 1.0, 1.0]))

#Realiza un producto elemento a elemento entre t_2 y su transpuesta.
print(t_2*t_2.t())

#Realiza un producto matricial entre t_2 y su transpuesta.
print(t_2@t_2.t())

#Selecciona todos los elementos de la t_2 que sean mayores que 0.5.
print(t_2>0.5)

#Crea un tensor booleano de la misma forma que el t_2, que sea True si el elemento es mayor que 0.5, y False en caso contrario.
t_2bool = torch.empty(3,3)
for i in range(t_2.size(0)):
  for j in range(t_2.size(1)):
    if t_2[i,j] > 0.5:
      t_2bool[i,j] = True
    else:
      t_2bool[i,j] = False
print(t_2bool)

#Cuenta cuantos elementos de t_2 son mayores de que la media de los elementos de t_2.
indices = torch.argwhere(t_2>t_2.mean())
print(indices.shape[0])

tensor([4.7833, 0.0530, 0.8470])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[9.7251e+01, 2.6665e+01, 2.2845e+01],
        [2.6665e+01, 2.8110e-03, 6.6811e-02],
        [2.2845e+01, 6.6811e-02, 9.7350e-01]])
tensor([[156.2861,  46.1751,  54.4252],
        [ 46.1751,  21.2682,  20.9454],
        [ 54.4252,  20.9454,  22.0860]])
tensor([[ True,  True,  True],
        [ True, False, False],
        [ True,  True,  True]])
tensor([[1., 1., 1.],
        [1., 0., 0.],
        [1., 1., 1.]])
5
